## Klassisk mekanikk numerisk øving 2
Eirik Jaccheri Høydalsvik, Hans Gløckner Giil

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#some constants
t0 = 0
g = 9.81
l, m = 1, 1
omega = np.sqrt(g / l) #NB; we will use omega as a helping constant, and thetaDot as the time-derivative of the angle

theta0 = 0.2
thetaDot0 = 0
u0 = np.array((theta0, thetaDot0))

#RK4, solves differential equation f. here: u0 = [theta, theta_dot]
def kutta_4(f, u0, t0, tNum, dt, q, F_d, omega_d):
    U = np.zeros( ((tNum),  u0.size) )
    U[0] = u0
    for i in range(1 ,tNum):
        F1 = f(i * dt, U[i-1], q, F_d, omega_d)
        F2 = f(i * dt + dt / 2, U[i-1] + dt  / 2 * F1, q, F_d, omega_d)
        F3 = f(i * dt + dt / 2, U[i-1] + dt / 2 * F2, q, F_d, omega_d)
        F4 = f(i * dt + dt, U[i-1] + dt * F3, q, F_d, omega_d)
        U[i] = U[i-1] + dt / 6 * (F1 + 2* F2 + 2 * F3 + F4)        
    return U

def f_driven (t, u, q, F_d , omega_d):#with driving force F(t) / m / l = D_d * sin(omega_d* t) 
    return np.array([u[1], - omega**2 *u[0]- q * u[1]  + F_d * np.sin(omega_d * t)])


def energy(u_arr): #Y is array containing vectors with theta, thetaDot at different times
    return pot(u_arr) + kin(u_arr)

def pot(u_arr):
    return  m * g * l * u_arr.T[0]**2 / 2

def kin(u_arr):
    return 0.5 * m * ( l * u_arr.T[1])**2

def work(u_arr): #difference between starting energy and energy at different times
    return energy(u_arr)[0] - energy(u_arr)

In [ ]:
#plot function for plotting theta(t)
def plot_theta(u0, t0, tNum, dt, q, plot_work, F_d = 0, omega_d = 0):
    fig, (ax1) = plt.subplots(1,1)
    #fig.set_size_inches(10, 8)
    t_arr = np.linspace(t0, t0 + dt * tNum, tNum)
    
    ax1.set_title("RK4, q =  " + str(q) , fontsize = 18)
    ax1.set_xlabel("Time", fontsize = 18)
    ax1.set_ylabel("Angle", fontsize = 18)
    #ax1.set(ylim = (-0.25, 0.25))
    u_arr = kutta_4(f_driven, u0, t0, tNum,  dt, q, F_d, omega_d)#array of [theta, theta_dot] values
    l1, = ax1.plot(t_arr, u_arr.T[0]) #UK.T gives [theta0, theta1 .... theta_tNum]
    if plot_work:
        #Bruker _tw for twin-axis
        ax1_tw = ax1.twinx()
        l1_tw, = ax1_tw.plot(t_arr, work(u_arr), color = "tab:red" ) #Replace energy with work to get work done by friction
        ax1_tw.set(ylim = (-1, 1))
        ax1_tw.set_ylabel("Work / J", fontsize = 18)
        ax1.legend((l1, l1_tw), ("Angle", "Friction work"), loc = "upper left", fontsize = 18)

    plt.tight_layout()
    return u_arr

def plot_theta_comparison(u0, t0, tNum, dt, q_arr, F_d = 0, omega_d = 0):
    num = len(q_arr)
    fig, ax = plt.subplots()
    #fig.set_size_inches(10, 8)
    t_arr = np.linspace(t0, t0 + dt * tNum, tNum)
    
    ax.set_title("RK4" , fontsize = 18)
    ax.set_xlabel("Time", fontsize = 18)
    ax.set_ylabel("Angle", fontsize = 18)
    fig.set_size_inches(12,8)
    for q in q_arr:
        u_arr = kutta_4(f_driven, u0, t0, tNum,  dt, q, F_d, omega_d)#array of [theta, theta_dot] values
        l1, = ax.plot(t_arr, u_arr.T[0]) #UK.T gives [theta0, theta1 .... theta_tNum]
    plt.show()
        

In [ ]:
def under(t,q,theta0,theta_dot):
    phi = np.arctan(np.sqrt(omega**2 - q**2/4)/ (theta_dot / theta0 + (q / 2)))
    A = theta0 / np.sin(phi)
    return A * np.exp(-q * t / 2) * np.sin(np.sqrt(omega**2 - q**2 / 4)*t + phi)

def critical(t,q,theta0,theta_dot):
    return (theta0 * (1 + q/2 * t) + theta_dot) * np.exp(-q * t / 2)

def over(t,q,theta0,theta_dot):
    lam_p, lam_m = -q/2 + np.sqrt(q**2/4 - omega**2), -q/2 - np.sqrt(q**2/4 - omega**2)
    A = (theta_dot / theta0 - lam_m) / (lam_p - lam_m)
    B = (theta_dot / theta0 - lam_p) / (lam_m - lam_p)
    return theta0 * (A * np.exp(lam_p * t) + B * np.exp(lam_m * t))

def forced(f,t,q,omega_d,F_d):
    A = F_d * (omega**2 - omega_d**2) / ((omega**2 - omega**2)**2 - (q * omega_d)**2)
    B = F_d * q * omega_d / ((omega**2 - omega_d**2)**2 - (q*omega_d)**2)
    return A * np.sin(omega_d * t) + B * np.cos(omega_d * t) + f(t,q,theta0 - B, -omega_d*A)


## Exercise 1 - Damped harmonic motion of pendulum
Figur xxxx viser de 3 tilfellene for theta0 = xx, theta_dot0 = xx, bla bla. Figur yy viser analytisk løsning.

In [ ]:
#bruk plot_theta til å plotte for 3 tilfeller : q <  2 * sqrt( g /l) (underdamped) , q > .. (verdamped),
# q  .. (critically damped). q_crit printet nedenfor.  Finn analytisk løsning og plot denne

t_num1, dt1 = 10000, 0.005
print(" Critical q value: ", np.sqrt(4 * omega **2))
#for q in np.arange(1, 10, 1):
q = 1
#    print( "q: ", q)
u_arr = plot_theta(u0,t0,t_num1, dt1, q ,plot_work = True)



## Exercise 2 - Forced harmonic damped pendulum
Figure YY shows the motion of the forced damped pendulum as a functon of time, for 5 different values of the driving frequency $\omega_D$.
Figure XX shows the amplitude as a function of $\omega = \sqrt{g / l}$ of the solution pendulum after some time, when the pendulum oscillates steadily. The largest amplitude happens when $ \omega_D = \omega$, in this case $\omega_D \approx 3.1$, the so called resonnance frequency. In figure XX+1 the amplitude from the analytic formula $ 
\begin{align}
\theta_0 = \frac{F_D}{\sqrt{(\Omega^2 - \Omega_D^2)^2 - (q \Omega_D)^2} }
\end{align}$ is plotted agains values of $\omega_D$. This analytic solution is plotted for the same 5 values of $q$ as the numerical solution.

In [ ]:
#plot for 5 different q-values
t_num2, dt2 = 1000, 0.05
F_d2, omega_d2 = 0.2, 3.1321
q2_arr = np.linspace(1, 7.5, 4 )
q2_arr = np.array((1, 10))
#for q2 in q2_arr:
#    u_arr = plot_theta(u0, t0, t_num2, dt2, q2, False, F_d2, omega_d2)
##q2 = print("Omega: ", round(omega, 4), "Omega_d : ", omega_d2)
##u_arr = plot_theta(u0, t0, t_num2, dt2, q2, False, F_d2, omega_d2)
#
plot_theta_comparison(u0, t0, t_num2, dt2, q2_arr, F_d2, omega_d2)

In [ ]:
def bode_plot(t_num, dt, omega, omega_d_num, q_num ):
    magnitude_arr = np.zeros(omega_d_num)
    
    x = np.linspace(0, omega_d_num-1, omega_d_num )
    omega_d_arr = np.linspace(0.1 * omega, 3 * omega, omega_d_num)
    time = t_num * dt
    q_arr = np.linspace(0.5, 5.5, q_num)
    
    fig, ax = plt.subplots(1,1)
    ax.set(xlim = (0, 6))
        
    for q in q_arr:
        i = 0
        
        for omega_d in omega_d_arr:
            u_arr = kutta_4(f_driven, u0, t0, t_num,  dt, q, F_d, omega_d)[:, 0] #take only theta-values
            m = (np.amax(u_arr[- int(2 * np.pi / omega_d / dt):]))
            #if i== 0:
            #     print("length u_arr: ", len(u_arr))
            #     print("slicing index:", - int(2 * np.pi / omega_d_arr[0] / dt) )
            magnitude_arr[i] = m  
            
            i +=1
            #if (i % 5) == 0:
            #   plot_theta(u0, t0, t_num2, dt2, q, False, F_d, omega_d)
        
        l1, = plt.plot(omega_d_arr, magnitude_arr)
    plt.show()
    return magnitude_arr

omega = np.sqrt(g / l)
F_d = 0.2
q = 1
t_num3, dt3 = 1000, 0.05
m_arr = bode_plot(t_num3, dt3, omega, 10, 10)

        

In [ ]:
#analytic bode plot
def anal_bode_plot(omega):
    return F_d / np.sqrt((omega**2 - omega_d**2)**2 + (q * omega_d)**2) 

omega_d = np.sqrt(g / l)
print(omega_d)

omega_d_arr2 = np.linspace(0, 6, 100)
fig2, ax2 = plt.subplots(1,1)
ax2.set(xlim = (0, 6))
for q in np.linspace(0.5, 5.5, 10):
    l1, = ax2.plot(omega_d_arr2, anal_bode_plot(omega_d_arr2))
    
                         
plt.show()
                         
